# NMF Model for Single Rank

In [14]:
from cvanmf import denovo
import pandas as pd
import plotnine as pn
import matplotlib.pyplot as plt

# Ignore plotnine's default size limits, as for big decompositions
# we will want some big plots
from plotnine.options import set_option
set_option("figure_format", False)

In [4]:
# Parameters for decomposition generation
# All left None unless provided, so use cvanmf defaults
model = "../output/4/model"

In [5]:
# Load the model
top_model = denovo.Decomposition.load(model)

In [6]:
# Some utility constants for sizing plots etc.

# for plot_modelfit_point
# The number of samples that will be represent in one inch of horizontal space
PLOT_MODELFIT_PPI = 20
# Maximum width allowed for plot
PLOT_MODELFIT_MAX_WIDTH = 20

# Limit on number of samples to plot for large studies
MAX_SAMPLES_PER_PLOT = 250

To edit or do further analysis on the model shown in this notebook, open it in Jupyter Lab or Notebook.

## Parameters


In [ ]:
# Print the parameters, except the input matrix
{k:v for k,v in top_model.parameters._asdict().items() if k != 'x'}

## Note on decompositions with many samples
The methods contained in these notebooks can be very slow when plotting 
many samples. As such, we slice the decomposition to contain only 250 samples.
You can manually select other subsets to plot, or generate complete plots as you
see fit. For very large studies, you may need to find other methods of 
visualisation.

In [8]:
sliced_model = top_model[:min(MAX_SAMPLES_PER_PLOT, top_model.h.shape[1]), :, :]

## Model Fit
We can measure how well the decomposition represents each sample, which we refer to as model fit.
This is calculated as the cosine similarity between the input values for a sample, and the reconstruced values (those in $WH$).
Cosine similarity for non-negative vectors falls between 0 (no relationship) and 1 (perfectly correlated).
A low median model fit indicates a poor model, however what is a poor median model fit will vary by field; however for genus level microbiome data  $<0.7$ seems an achievable median model fit.

Some samples having poor model fit is not neccesarily a disqualifying factor, depending on your use case.
Those samples may be in an unusual state for some reason (such as clinical factors), or represent interesting unusual deviations from a typical state.

### Distribution of Model Fit
Below is a histogram of the model fit across all samples, with a vertical line indicating median.

In [ ]:
top_model.plot_modelfit() + pn.theme(figure_size=(6,4)) + pn.ggtitle("Distribution of Model Fit")

### Model Fit Per Sample
Below, the model fit is shown for each sample as a dot. Those below a threshold value are shown in red.
This threshold was determined for our Enterosignatures model, and may not be suitable for other data.
To use a different value, set the `threshold` parameter in `plot_modelfit_point()`.

In [ ]:
# Make the plot wider when there are more samples
modelfit_point_width = min((sliced_model.h.shape[1] / PLOT_MODELFIT_PPI) + 1, PLOT_MODELFIT_MAX_WIDTH)
# Customise width and text 
(
    sliced_model.plot_modelfit_point() + 
    pn.theme(
        figure_size=(modelfit_point_width, 2), 
        axis_text_x=pn.element_text(size=7)
    ) +
    pn.ggtitle("Model Fit")
)

# Feature Weights
Looking at the feature weights in each signature can help interpret the meaning of each signature.
Here we should the relative weight of features in each signature, meaning that the values in each signature are scaled to sum to 1.
The plot shows any features whose weight make up at least 4% of the total in the signature, this can be changed using the `threshold` parameter in `plot_feature_weight`.
Feature labels are trimmed by default to a maximum of 40 characters, but you can define some other method of truncating, or none using `label_fn`.

In [ ]:
def truncate_name(x):
    if len(x) <= 40:
        return x
    return x[0:29] + "..." + x[-18:]
# Allow 1 inch per signature
feature_weight_width = top_model.w.shape[1] + 3
(
    top_model.plot_feature_weight(
        label_fn=truncate_name
    ) + 
    pn.theme(figure_size=(feature_weight_width, 6)) +
    pn.ggtitle("Relative Signature Weight")
)

# PCoA
Here PCoA is performed on the scaled signature weight matrix. For low rank models, this often doesn't show much, however it is more useful at higher ranks.
PCoA is carried out using Bray-Curtis dissimilarity, but can be customised - see documentation for `pcoa` and `plot_pcoa` methods of `Decomposition`.
This plot is often most useful when combined with metadata - you can use the `color` and `shape` arguments to provide categorical metadata to determine the shape and colour of points.

In [ ]:
(
    sliced_model.plot_pcoa() +
    pn.ggtitle("PCoA on Relative Signature Weight")
)

### Relative Weight Plot
This represents the weight of each signature in each sample, with the weights in each sample scaled so that they sum to 1.
We also append the model fit point plot to the top.
This is the only figure which is not returned a `plotnine.ggplot` object, so we have to define widths etc in the call.
This plot can be quite expensive to produce when there are a lot of signatures or samples. 

In [ ]:
legend_cols = (sliced_model.w.shape[1] // 6) + 1
sliced_model.plot_relative_weight(
    heights=[0.3, 0.7, 0.3], width=modelfit_point_width, point_size=.4,
    legend_cols_v=legend_cols,
).render()
plt.show()

# Additional analyses available

## Against metadata
Methods are provided to plot signatures against metadata, as well as some simple statistical tests.
* `plot_metadata` takes a dataframe of metadata for each sample, and makes box plots for categorical and scatter plots for continuous medata. For categorical metadata, nonparametric statistical tests will be carried out.
* `univariate_tests` carries out either Mann-Whitney U tests or Kruskal-Wallis tests against relative signature weights for a dataframe of categorical metadata.

## Characterise samples
* `representative_signatures` indicates which signatures are representative of sample, found by taking all those which sum up to 90% of the total weight in the sample (though this threshold can be specified)